# core

> Helper Functions for the analysis

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import subprocess
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

from pycirclize import Circos
from pycirclize.parser import Gff
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
#| export

def count_variants(vcf_file):
    """Count the number of variants in a VCF file using subprocess."""
    if vcf_file.endswith('.gz'):
        cmd = f"bcftools view -H {vcf_file} | wc -l"
    else:
        cmd = f"grep -v '^#' {vcf_file} | wc -l"
    
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return int(result.stdout.strip())

def filter_variants():
    # Define input and output VCF files
    INPUT_VCF = "../data/freebayes.annotated_pc1.vcf.gz"
    QUAL_FILTERED_VCF = "../data/filtered_qual.vcf"
    DP_FILTERED_VCF = "../data/filtered_dp.vcf"
    SNP_FILTERED_VCF = "../data/filtered_snp.vcf"
    FINAL_VCF = "../data/filtered_final.vcf"
    
    print("======================================")
    print("Starting Variant Filtering Process")
    print("======================================")
    
    # Count initial number of variants
    START_COUNT = count_variants(INPUT_VCF)
    print(f"Total variants before filtering: {START_COUNT}")
    
    # Step 1: Filter out low-quality variants (QUAL < 30)
    subprocess.run(f"bcftools filter -e 'QUAL < 30' {INPUT_VCF} -o {QUAL_FILTERED_VCF}", shell=True)
    QUAL_FILTERED_COUNT = count_variants(QUAL_FILTERED_VCF)
    print(f"Stage 1: QUAL filtering: {START_COUNT - QUAL_FILTERED_COUNT} Variants removed and {QUAL_FILTERED_COUNT} variants left")

    # Step 2: Filter variants based on per-sample depth (FORMAT/DP < 10 or > 150)
    subprocess.run(f"bcftools view -i 'FMT/DP >= 30 & FMT/DP <= 150' {QUAL_FILTERED_VCF} -o {DP_FILTERED_VCF}", shell=True)
    DP_FILTERED_COUNT = count_variants(DP_FILTERED_VCF)
    print(f"Stage 2: FORMAT/DP filtering, DP >= 30 & DP <= 150: {QUAL_FILTERED_COUNT - DP_FILTERED_COUNT} Variants removed and {DP_FILTERED_COUNT} variants left")

    # Step 3: Retain SNPs and indels (Remove other variant types if any)
    subprocess.run(f"bcftools view -v snps,indels {DP_FILTERED_VCF} -o {SNP_FILTERED_VCF}", shell=True)
    SNP_FILTERED_COUNT = count_variants(SNP_FILTERED_VCF)
    print(f"Stage 3: After keeping SNPs and indels: {DP_FILTERED_COUNT - SNP_FILTERED_COUNT} Variants removed and {SNP_FILTERED_COUNT} variants left")

    # Rename final output
    os.rename(SNP_FILTERED_VCF, FINAL_VCF)
    FINAL_COUNT = count_variants(FINAL_VCF)


In [ ]:
#| export

def read_vcf(vcf_file):
    """Reads a VCF file, automatically detecting the header and using correct column names."""
    
    # Find the header line dynamically
    with open(vcf_file, "r") as f:
        for line in f:
            if line.startswith("#CHROM"):
                header = line.strip().split("\t")  # Extract column names
                break  # Stop searching after finding header
    
    # Read VCF using Pandas, skipping comment lines
    df = pd.read_csv(vcf_file, sep="\t", comment="#", header=None, names=header)
    return df


In [ ]:
#| export

def find_index(format_value, field):
    """Finds the index of a specific field (e.g., RO, AO, DP) in the FORMAT column."""
    item_list = format_value.split(':')
    return item_list.index(field) if field in item_list else None

def expand_multiallelic_variants(df_vcf):
    """
    Extracts allele counts (RO, AO, DP) for each sample from the VCF DataFrame
    and expands multi-allelic variants into separate rows.
    """

    # Extract sample names (everything after FORMAT column)
    sample_names = df_vcf.columns[9:]  # Skip CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO, FORMAT

    # Find the index positions of RO, AO, and DP in the FORMAT field
    format_example = df_vcf['FORMAT'].values[0]  # Take the first row as an example
    ro_index = find_index(format_example, 'RO')
    ao_index = find_index(format_example, 'AO')
    dp_index = find_index(format_example, 'DP')

    # Ensure indices exist
    if None in [ro_index, ao_index, dp_index]:
        raise ValueError("RO, AO, or DP field not found in FORMAT column.")

    # Initialize an empty list to store expanded rows
    expanded_rows = []

    for _, row in df_vcf.iterrows():
        # Split ALT alleles (multi-allelic sites will have multiple ALT values)
        alt_alleles = row['ALT'].split(',')
        info_type = row['INFO'].split('TYPE=')[1].split(';')[0].split(',')
        
        # Process each ALT allele separately
        for i, (alt, inty )in enumerate(zip(alt_alleles,info_type)):
            new_row = {
                "#CHROM": row["#CHROM"],
                "POS": row["POS"],
                "REF": row["REF"],
                "ALT": alt,  # Assign each alternate allele to a separate row
                "INFO_TYPE": inty
            }

            for sample in sample_names:
                # Split FORMAT fields for the sample
                sample_values = row[sample].split(':')
                
                # Extract and store RO and DP
                new_row[f"RO_{sample}"] = int(sample_values[ro_index]) if sample_values[ro_index] != '.' else 0
                new_row[f"DP_{sample}"] = int(sample_values[dp_index]) if sample_values[dp_index] != '.' else 0

                # Extract AO for the specific ALT allele
                ao_values = sample_values[ao_index].split(',')  # Multiple values for multiple ALT alleles
                new_row[f"AO_{sample}"] = int(ao_values[i]) if i < len(ao_values) and ao_values[i] != '.' else 0

            # Append expanded row
            expanded_rows.append(new_row)

    # Convert list of dictionaries into DataFrame
    expanded_df = pd.DataFrame(expanded_rows)

    return expanded_df

In [ ]:
#| export

def compute_frequencies(df_counts):
    """
    Computes allele frequency (AF = AO / DP) for each sample in the dataset.
    """

    # Extract sample names from AO columns
    sample_names = [col.replace("AO_", "") for col in df_counts.columns if col.startswith("AO_")]

    # Create a new DataFrame to store allele frequencies
    df_af = df_counts[['#CHROM', 'POS', 'REF', 'ALT']].copy()

    for sample in sample_names:
        ao_col = f"AO_{sample}"
        dp_col = f"DP_{sample}"
        af_col = f"AF_{sample}"
        df_af[af_col] = df_counts[ao_col] / df_counts[dp_col] #(df_counts[ro_col] + df_counts[ao_col])

    return df_af


In [ ]:
#| export
def extract_first_ann(info_field):
    """
    Extract the first ANN annotation from a VCF INFO field.
    
    Parameters:
    -----------
    info_field : str
        The INFO field from a VCF file
    
    Returns:
    --------
    dict
        A dictionary containing the variant type, impact, and gene ID,
        or None if no ANN field is found
    """
    # Check if there's an ANN field
    if 'ANN=' not in info_field:
        return None
    
    # Extract the ANN part
    ann_start = info_field.find('ANN=')
    # Take everything after "ANN="
    ann_content = info_field[ann_start + 4:]
    
    # If there are other fields after ANN, cut them off
    if ';' in ann_content:
        ann_content = ann_content.split(';')[0]
    
    # Split by comma to get individual annotations
    annotations = ann_content.split(',')
    
    # Get the first annotation
    first_ann = annotations[0]
    
    # Split by pipe (|) to get annotation fields
    ann_fields = first_ann.split('|')
    
    # Create result dictionary
    # Standard VCF ANN format: Allele | Annotation | Impact | Gene_Name | ...
    if len(ann_fields) >= 4:
        result = {
            'allele': ann_fields[0],
            'type': ann_fields[1],
            'impact': ann_fields[2],
            'gene_id': ann_fields[3]
        }
        return result
    else:
        return None



def add_ann_info_to_df(df, info_column='INFO'):
    """
    Extract the first ANN annotation from the INFO field and add as separate columns to a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing VCF data with an INFO column
    info_column : str, default='INFO'
        Name of the column containing the INFO field
        
    Returns:
    --------
    pandas.DataFrame
        The original DataFrame with additional columns for variant type, impact, and gene ID
    """
    # Create new columns with None values
    df['variant_type'] = None
    df['impact'] = None
    df['gene_id'] = None
    df['allele'] = None
    
    # Process each row
    for idx, row in df.iterrows():
        info = row[info_column]
        ann_data = extract_first_ann(info)
        
        if ann_data:
            df.loc[idx, 'variant_type'] = ann_data['type']
            df.loc[idx, 'impact'] = ann_data['impact']
            df.loc[idx, 'gene_id'] = ann_data['gene_id']
            df.loc[idx, 'allele'] = ann_data['allele']
    
    return df

# Example usage
# Assuming you have a DataFrame 'vcf_df' with a column 'INFO' containing VCF INFO fields
# vcf_df = add_ann_info_to_df(vcf_df)

In [ ]:
# Test with the example
info_field = 'AB=0;ABP=0;AC=1;AF=0.142857;AN=7;AO=248;CIGAR=1X;DP=664;DPB=664;DPRA=0;EPP=13.1322;EPPR=3.03118;GTI=0;LEN=1;MEANALT=1;MQM=60;MQMR=60;NS=7;NUMALT=1;ODDS=61.2192;PAIRED=0.991935;PAIREDR=0.995192;PAO=0;PQA=0;PQR=0;PRO=0;QA=8364;QR=13932;RO=416;RPL=137;RPP=8.92931;RPPR=9.04449;RPR=111;RUN=1;SAF=122;SAP=3.15039;SAR=126;SRF=210;SRP=3.09382;SRR=206;TYPE=snp;technology.illumina=1;ANN=G|synonymous_variant|LOW|cgd1_340|cgd1_340|transcript|cgd1_340-RA|protein_coding|1/1|c.357A>G|p.Ser119Ser|570/1483|357/1194|119/397||,G|upstream_gene_variant|MODIFIER|cgd1_320|cgd1_320|transcript|cgd1_320-RA|protein_coding||c.-4699T>C|||||4675|,G|upstream_gene_variant|MODIFIER|cgd1_330|cgd1_330|transcript|cgd1_330-RA|protein_coding||c.-1174T>C|||||1095|,G|upstream_gene_variant|MODIFIER|cgd1_350|cgd1_350|transcript|cgd1_350-RA|protein_coding||c.-1490A>G|||||1323|,G|downstream_gene_variant|MODIFIER|cgd1_360|cgd1_360|transcript|cgd1_360-RA|protein_coding||c.*4665T>C|||||4665|'
result = extract_first_ann(info_field)
print(result)
# Should output: {'allele': 'G', 'type': 'synonymous_variant', 'impact': 'LOW', 'gene_id': 'cgd1_340'}

{'allele': 'G', 'type': 'synonymous_variant', 'impact': 'LOW', 'gene_id': 'cgd1_340'}


REF ALT     AF_M7     AF_M5     AF_M4     AF_M6     AF_C3  \
#CHROM   POS                                                                 
CM000429 76625     A   G  0.269231  0.350000  0.404762  0.314607  0.566265   
         82019     A   T  0.908163  0.705882  0.779070  0.833333  0.628571   
         82192     G   A  0.894231  0.701493  0.666667  0.745098  0.471698   
         106518    C   G  1.000000  1.000000  1.000000  1.000000  1.000000   
         116718    C   A  1.000000  1.000000  1.000000  0.989796  1.000000   
...               ..  ..       ...       ...       ...       ...       ...   
CM000436 1198389   T   A  1.000000  0.981818  1.000000  1.000000  0.987654   
         1198626   A   C  1.000000  1.000000  1.000000  1.000000  1.000000   
         1198636   G   A  1.000000  1.000000  0.988889  1.000000  1.000000   
         1198801   C   G  1.000000  1.000000  1.000000  1.000000  1.000000   
         1210605   C   T  0.827586  0.718310  0.711111  0.781818  0.673684   

                     AF_C2     AF_C1  \
#CHROM   POS                           
CM000429 76625    0.303571  0.437500   
         82019    0.617284  0.690141   
         82192    0.619565  0.602410   
         106518   1.000000  1.000000   
         116718   1.000000  1.000000   
...                    ...       ...   
CM000436 1198389  1.000000  1.000000   
         1198626  1.000000  0.987013   
         1198636  1.000000  1.000000   
         1198801  1.000000  1.000000   
         1210605  0.735294  0.747573   

                                                               INFO  \
#CHROM   POS                                                          
CM000429 76625    AB=0;ABP=0;AC=1;AF=0.142857;AN=7;AO=248;CIGAR=...   
         82019    AB=0;ABP=0;AC=7;AF=1;AN=7;AO=410;CIGAR=1X;DP=5...   
         82192    AB=0;ABP=0;AC=6;AF=0.857143;AN=7;AO=398;CIGAR=...   
         106518   AB=0;ABP=0;AC=7;AF=1;AN=7;AO=530;CIGAR=1X;DP=5...   
         116718   AB=0;ABP=0;AC=7;AF=1;AN=7;AO=525;CIGAR=1X;DP=5...   
...                                                             ...   
CM000436 1198389  AB=0;ABP=0;AC=7;AF=1;AN=7;AO=531;CIGAR=1X;DP=5...   
         1198626  AB=0;ABP=0;AC=7;AF=1;AN=7;AO=626;CIGAR=1X;DP=6...   
         1198636  AB=0;ABP=0;AC=7;AF=1;AN=7;AO=650;CIGAR=1X;DP=6...   
         1198801  AB=0;ABP=0;AC=7;AF=1;AN=7;AO=447;CIGAR=1X;DP=4...   
         1210605  AB=0;ABP=0;AC=7;AF=1;AN=7;AO=464;CIGAR=1X;DP=6...   

                           variant_type    impact    gene_id allele  
#CHROM   POS                                                         
CM000429 76625       synonymous_variant       LOW   cgd1_340      G  
         82019       synonymous_variant       LOW   cgd1_360      T  
         82192         missense_variant  MODERATE   cgd1_360      A  
         106518        missense_variant  MODERATE   cgd1_493      G  
         116718   upstream_gene_variant  MODIFIER   cgd1_520      A  
...                                 ...       ...        ...    ...  
CM000436 1198389       missense_variant  MODERATE  cgd8_4800      A  
         1198626    3_prime_UTR_variant  MODIFIER  cgd8_4800      C  
         1198636    3_prime_UTR_variant  MODIFIER  cgd8_4800      A  
         1198801    3_prime_UTR_variant  MODIFIER  cgd8_4800      G  
         1210605     synonymous_variant       LOW  cgd8_4840      T  

[515 rows x 14 columns]

In [ ]:
#| export

def mod_hist_legend(ax, title=False):
    """
    Creates a cleaner legend for histogram plots by using line elements instead of patches.
    
    Motivation:
    - Default histogram legends show rectangle patches which can be visually distracting
    - This function creates a more elegant legend with simple lines matching histogram edge colors
    - Positions the legend outside the plot to avoid overlapping with data
    
    Parameters:
    -----------
    ax : matplotlib.axes.Axes
        The axes object containing the histogram(s)
    title : str or bool, default=False
        Optional title for the legend. If False, no title is displayed
        
    Returns:
    --------
    None - modifies the axes object in place
    """
    # Extract the current handles and labels from the plot
    handles, labels = ax.get_legend_handles_labels()
    
    # Create new line handles that match the edge colors of histogram bars
    # This produces a cleaner, more minimal legend appearance
    new_handles = [matplotlib.lines.Line2D([], [], c=h.get_edgecolor()) for h in handles]
    
    # Create the legend with custom positioning
    # - Places legend outside the plot (to the right) to avoid obscuring the data
    # - Centers the legend vertically for better visual balance
    ax.legend(handles=new_handles, 
              labels=labels, 
              title=title,
              loc='center left', 
              bbox_to_anchor=(1, 0.5))


def clean_axes(ax, offset=10):
    """
    Customizes a matplotlib axes by removing top and right spines,
    and creating a broken axis effect where x and y axes don't touch.
    
    Parameters:
    -----------
    ax : matplotlib.axes.Axes
        The axes object to customize
    offset : int, default=10
        The amount of offset/gap between the x and y axes in points
        
    Returns:
    --------
    ax : matplotlib.axes.Axes
        The same axes object, modified in place
    """
    # Remove the top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Make the remaining spines gray for a more subtle look
    ax.spines['left'].set_color('gray')
    ax.spines['bottom'].set_color('gray')
    
    # Create the broken axis effect
    # Move the bottom spine up by offset points
    #ax.spines['bottom'].set_position(('outward', offset))
    
    # Move the left spine right by offset points
    ax.spines['left'].set_position(('outward', offset))
    
    # Return the modified axes
    return ax

In [ ]:
#| export

def make_circos_plot(data):
    
    seqid2size = {
        'CM000429': 875659,
        'CM000430': 985969,
        'CM000431': 1099352,
        'CM000432': 1104417,
        'CM000433': 1080900,
        'CM000434': 1332857,
        'CM000435': 1278458,
        'CM000436': 1344712
    }

    color_dict = {'M': 'blue', 'C': 'magenta'}

    circos = Circos(seqid2size, space=3, start=-83, end=265, endspace=False)
    circos.text("C. parvum IowaII", r=5, size=18, font={'style': 'italic'})
    
    m_samples = ['AF_M7', 'AF_M5', 'AF_M6', 'AF_M4']
    c_samples = ['AF_C3', 'AF_C2', 'AF_C1']
    
    for sector in circos.sectors:
        sector.text(sector.name[-3:])
        
        m_track = sector.add_track((80, 100))
        m_track.xticks_by_interval(200000, show_label=False)
        m_track.axis()
    
        
        c_track = sector.add_track((55, 75))
        c_track.xticks_by_interval(200000, show_label=False)
        c_track.axis()
        
        # Plot scatter points for each sample group
        for sample in m_samples:
            color = color_dict[sample[3]]
            subset = data[(data['#CHROM'] == sector.name) & (data[sample] > 0)]
            m_track.scatter(
                x=subset['POS'].values,
                y=subset[sample].values,
                c=color,
                s=3,
                vmin=0,
                vmax=1,
                alpha=0.3,
            )
        
        
        for sample in c_samples:
            color = color_dict[sample[3]]
            subset = data[(data['#CHROM'] == sector.name) & (data[sample] > 0)]
            c_track.scatter(
                x=subset['POS'].values,
                y=subset[sample].values,
                c=color,
                s=3,
                vmin=0,
                vmax=1,
                alpha=0.3,
            )

        
        # Optional: Add labels 
        if sector.name == 'CM000429':
            m_track.yticks([0, 1], ["0", "1"], side="left")
            c_track.yticks([0, 1], ["0", "1"], side="left")

            
            circos.text("Cow", r=c_track.r_center, deg=-90, color="magenta")
            circos.text("Mouse", r=m_track.r_center, deg=-90, color="blue")
    
    circos.plotfig()
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()